<a href="https://colab.research.google.com/github/Diggi14/project_Property2/blob/main/feature_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
!git clone https://github.com/Diggi14/project_Property2.git

fatal: destination path 'project_Property2' already exists and is not an empty directory.


In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [22]:
df=pd.read_csv('/content/project_Property2/cleaned_data3.csv')

In [23]:
df.head(3)

,PREFERENCE,PROPERTY_TYPE,BEDROOM_NUM,BATHROOM_NUM,BALCONY_NUM,PRICE_PER_UNIT_AREA,FURNISH,AGE,FLOOR_NUM,SUPERBUILTUP_SQFT,PRICE,LOCALITY_WO_CITY,stories
0,S,Residential Apartment,4.0,4.0,4.0,8766.0,Semi-furnished,5-10,14.0,3434.0,2.63,Sector 84,single
1,S,Residential Apartment,4.0,4.0,3.0,21176.0,Semi-furnished,1-5,7.0,2870.0,3.60,Sector 81,single
2,S,Residential Apartment,3.0,3.0,3.0,13740.0,Semi-furnished,1-5,14.0,2802.0,3.85,Sector 112,single


In [24]:
df.duplicated().sum()

np.int64(3)

In [25]:
df.drop_duplicates(inplace=True)

In [26]:
df['FLOOR_NUM'].describe()

,FLOOR_NUM
count,9330.000000
mean,6.804716
std,6.210222
min,-1.000000
25%,2.000000
50%,5.000000
75%,10.000000
max,48.000000


In [27]:
def categotize_floor(val):
  if(val<=2):
    return 'low'
  elif(val<=10):
    return 'mid'
  else:
    return 'high'

In [28]:
df['FLOOR_NUM']=df['FLOOR_NUM'].apply(categotize_floor)

In [29]:
df.head(2)

,PREFERENCE,PROPERTY_TYPE,BEDROOM_NUM,BATHROOM_NUM,BALCONY_NUM,PRICE_PER_UNIT_AREA,FURNISH,AGE,FLOOR_NUM,SUPERBUILTUP_SQFT,PRICE,LOCALITY_WO_CITY,stories
0,S,Residential Apartment,4.0,4.0,4.0,8766.0,Semi-furnished,5-10,high,3434.0,2.63,Sector 84,single
1,S,Residential Apartment,4.0,4.0,3.0,21176.0,Semi-furnished,1-5,mid,2870.0,3.60,Sector 81,single


In [30]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
df_encode=df.copy()
col=df.select_dtypes(include='object').columns
for i in col:
  df_encode[i]=le.fit_transform(df_encode[i])

In [31]:
df_encode.head(2)

,PREFERENCE,PROPERTY_TYPE,BEDROOM_NUM,BATHROOM_NUM,BALCONY_NUM,PRICE_PER_UNIT_AREA,FURNISH,AGE,FLOOR_NUM,SUPERBUILTUP_SQFT,PRICE,LOCALITY_WO_CITY,stories
0,1,2,4.0,4.0,4.0,8766.0,1,3,0,3434.0,2.63,136,1
1,1,2,4.0,4.0,3.0,21176.0,1,1,2,2870.0,3.60,132,1


In [32]:
corr1=df_encode.corr()['PRICE'].sort_values(ascending=False).iloc[1:].reset_index().rename(columns={'index':'feature','PRICE':'correlation'})

In [33]:
corr1

,feature,correlation
0,SUPERBUILTUP_SQFT,0.758708
1,PRICE_PER_UNIT_AREA,0.746367
2,BATHROOM_NUM,0.593832
3,BEDROOM_NUM,0.527969
4,BALCONY_NUM,0.206935
5,PREFERENCE,0.009958
6,AGE,-0.000130
7,FLOOR_NUM,-0.106061
8,FURNISH,-0.152328
9,PROPERTY_TYPE,-0.165310


In [34]:
x=df_encode.drop('PRICE',axis=1)
y=df_encode['PRICE']

In [35]:
from sklearn.ensemble import RandomForestRegressor
rf=RandomForestRegressor()
rf.fit(x,y)
feat_imp=pd.DataFrame({'feature':x.columns,'importance':rf.feature_importances_}).sort_values(by='importance',ascending=False)

In [36]:
feat_imp

,feature,importance
9,SUPERBUILTUP_SQFT,0.568910
5,PRICE_PER_UNIT_AREA,0.417723
10,LOCALITY_WO_CITY,0.006087
3,BATHROOM_NUM,0.002218
2,BEDROOM_NUM,0.001306
7,AGE,0.001252
4,BALCONY_NUM,0.000870
8,FLOOR_NUM,0.000558
6,FURNISH,0.000533
1,PROPERTY_TYPE,0.000429


In [37]:
from sklearn.ensemble import GradientBoostingRegressor
gbr = GradientBoostingRegressor(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=3,
    random_state=42
)

gbr.fit(x,y)


GradientBoostingRegressor(learning_rate=0.05, n_estimators=300, random_state=42)

In [38]:
feat_imp2=pd.DataFrame({'feature':x.columns,'importance':gbr.feature_importances_}).sort_values(by='importance',ascending=False)

In [39]:
feat_imp2

,feature,importance
9,SUPERBUILTUP_SQFT,0.559038
5,PRICE_PER_UNIT_AREA,0.432887
3,BATHROOM_NUM,0.003852
10,LOCALITY_WO_CITY,0.003000
1,PROPERTY_TYPE,0.000337
7,AGE,0.000322
2,BEDROOM_NUM,0.000266
8,FLOOR_NUM,0.000120
4,BALCONY_NUM,0.000061
6,FURNISH,0.000055


In [40]:

from sklearn.inspection import permutation_importance
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)
rf=RandomForestRegressor(n_estimators=300,random_state=42,n_jobs=-1)
rf.fit(x_train,y_train)
perm = permutation_importance(rf, x, y, n_repeats=30, random_state=42, n_jobs=-1)
perm_df = pd.DataFrame({
    'feature': x_train.columns,
    'importance_mean': perm.importances_mean,
}).sort_values(by='importance_mean', ascending=False)

perm_df


,feature,importance_mean
9,SUPERBUILTUP_SQFT,0.839390
5,PRICE_PER_UNIT_AREA,0.787755
10,LOCALITY_WO_CITY,0.012102
3,BATHROOM_NUM,0.004103
7,AGE,0.001475
4,BALCONY_NUM,0.001055
2,BEDROOM_NUM,0.000928
1,PROPERTY_TYPE,0.000436
8,FLOOR_NUM,0.000374
6,FURNISH,0.000353


In [41]:

from sklearn.feature_selection import RFE

rf = RandomForestRegressor(n_estimators=300,random_state=42,n_jobs=-1)

rfe = RFE(estimator=rf,n_features_to_select=x.shape[1],step=1)
select_label=rfe.fit(x,y)
selected_features = x.columns[rfe.support_]
selected_coff=select_label.estimator_.feature_importances_

rfe_ranking = pd.DataFrame({
    'feature': x.columns,
    'selected_coff': selected_coff
}).sort_values(by='selected_coff',ascending=False)

rfe_ranking

,feature,selected_coff
9,SUPERBUILTUP_SQFT,0.568904
5,PRICE_PER_UNIT_AREA,0.417607
10,LOCALITY_WO_CITY,0.006159
3,BATHROOM_NUM,0.002129
2,BEDROOM_NUM,0.001334
7,AGE,0.001304
4,BALCONY_NUM,0.000925
8,FLOOR_NUM,0.000547
6,FURNISH,0.000520
1,PROPERTY_TYPE,0.000453


In [42]:
import shap
rf = RandomForestRegressor(
    n_estimators=100,
    random_state=42,
    n_jobs=-1
)
rf.fit(x,y)

RandomForestRegressor(n_jobs=-1, random_state=42)

In [44]:
explainer = shap.TreeExplainer(rf)
shap_values = explainer.shap_values(x)
shap_importance = pd.DataFrame({
    'feature': x.columns,
    'mean_abs_shap': np.abs(shap_values).mean(axis=0)
}).sort_values(by='mean_abs_shap', ascending=False)

shap_importance

,feature,mean_abs_shap
9,SUPERBUILTUP_SQFT,0.970384
5,PRICE_PER_UNIT_AREA,0.824392
10,LOCALITY_WO_CITY,0.039631
3,BATHROOM_NUM,0.012486
2,BEDROOM_NUM,0.008529
7,AGE,0.005639
4,BALCONY_NUM,0.004642
6,FURNISH,0.002937
1,PROPERTY_TYPE,0.002914
8,FLOOR_NUM,0.002352


In [45]:
rel=corr1.merge(feat_imp,on='feature').merge(feat_imp2,on='feature').merge(perm_df,on='feature').merge(rfe_ranking,on='feature').merge(shap_importance,on='feature')

In [46]:
from sklearn.preprocessing import MinMaxScaler
mn=MinMaxScaler()
col=[ 'correlation', 'importance_x', 'importance_y','importance_mean', 'selected_coff', 'mean_abs_shap']
rel_copy=rel.copy()
rel_copy[col]=mn.fit_transform(rel_copy[col])

In [47]:
rel_copy

,feature,correlation,importance_x,importance_y,importance_mean,selected_coff,mean_abs_shap
0,SUPERBUILTUP_SQFT,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,PRICE_PER_UNIT_AREA,0.987783,0.734242,0.774333,0.938484,0.734047,0.849531
2,BATHROOM_NUM,0.836787,0.003858,0.006847,0.004874,0.003708,0.012726
3,BEDROOM_NUM,0.771588,0.002256,0.000432,0.001092,0.002310,0.008648
4,BALCONY_NUM,0.453790,0.001490,0.000066,0.001243,0.001592,0.004642
5,PREFERENCE,0.258800,0.000000,0.000027,0.000000,0.000000,0.000000
6,AGE,0.248813,0.002160,0.000533,0.001744,0.002259,0.005669
7,FLOOR_NUM,0.143950,0.000941,0.000171,0.000432,0.000927,0.002282
8,FURNISH,0.098150,0.000897,0.000055,0.000407,0.000879,0.002884
9,PROPERTY_TYPE,0.085299,0.000715,0.000559,0.000506,0.000762,0.002861


In [48]:
rel_copy['importance_row_mean'] = rel_copy[
    ['correlation','importance_x', 'importance_y', 'importance_mean','selected_coff', 'mean_abs_shap']
].mean(axis=1)

In [49]:
rel_copy

,feature,correlation,importance_x,importance_y,importance_mean,selected_coff,mean_abs_shap,importance_row_mean
0,SUPERBUILTUP_SQFT,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,PRICE_PER_UNIT_AREA,0.987783,0.734242,0.774333,0.938484,0.734047,0.849531,0.836403
2,BATHROOM_NUM,0.836787,0.003858,0.006847,0.004874,0.003708,0.012726,0.144800
3,BEDROOM_NUM,0.771588,0.002256,0.000432,0.001092,0.002310,0.008648,0.131054
4,BALCONY_NUM,0.453790,0.001490,0.000066,0.001243,0.001592,0.004642,0.077137
5,PREFERENCE,0.258800,0.000000,0.000027,0.000000,0.000000,0.000000,0.043138
6,AGE,0.248813,0.002160,0.000533,0.001744,0.002259,0.005669,0.043530
7,FLOOR_NUM,0.143950,0.000941,0.000171,0.000432,0.000927,0.002282,0.024784
8,FURNISH,0.098150,0.000897,0.000055,0.000407,0.000879,0.002884,0.017212
9,PROPERTY_TYPE,0.085299,0.000715,0.000559,0.000506,0.000762,0.002861,0.015117


In [50]:
rel_copy.drop(['correlation','importance_x', 'importance_y', 'importance_mean','selected_coff', 'mean_abs_shap'],axis=1,inplace=True)

In [51]:
rel_copy.sort_values(by='importance_row_mean',ascending=False)

,feature,importance_row_mean
0,SUPERBUILTUP_SQFT,1.000000
1,PRICE_PER_UNIT_AREA,0.836403
2,BATHROOM_NUM,0.144800
3,BEDROOM_NUM,0.131054
4,BALCONY_NUM,0.077137
6,AGE,0.043530
5,PREFERENCE,0.043138
7,FLOOR_NUM,0.024784
10,LOCALITY_WO_CITY,0.019414
8,FURNISH,0.017212


In [53]:
from sklearn.model_selection import cross_val_score
rf=RandomForestRegressor(n_estimators=300,random_state=42,n_jobs=-1)
scores=cross_val_score(rf,x,y,scoring='r2',cv=10)

In [54]:
scores.mean()

np.float64(0.9781774664508042)

In [55]:
rf=RandomForestRegressor(n_estimators=300,random_state=42,n_jobs=-1)
scores=cross_val_score(rf,x.drop(['stories','PROPERTY_TYPE'],axis=1),y,scoring='r2',cv=10)
scores.mean()

np.float64(0.978097031272438)

In [56]:
rf=RandomForestRegressor(n_estimators=300,random_state=42,n_jobs=-1)
scores=cross_val_score(rf,x.drop(['stories','PREFERENCE'],axis=1),y,scoring='r2',cv=10)
scores.mean()

np.float64(0.9782251912769088)

In [57]:
rf=RandomForestRegressor(n_estimators=300,random_state=42,n_jobs=-1)
scores=cross_val_score(rf,x.drop(['stories','PREFERENCE','PROPERTY_TYPE'],axis=1),y,scoring='r2',cv=10)
scores.mean()

np.float64(0.9781429386787336)

In [58]:
export_df=df.drop(['stories','PREFERENCE','PROPERTY_TYPE'],axis=1)

In [60]:
export_df.to_csv("model_trainer.csv")